## Pose_Estimation

### Install requierd libraries

In [ ]:
'''Install requiered dependencies/libraries
-pip install numpy
-pip install opencv-python
-pip install scikit-learn
-pip install tensorflow
-pip install keras
-pip install hmmlearn
-pip install matplotlib
-pip install matplotlib
'''

### Import installed and required python libraries

In [16]:
#Import requiered  installed libraries
import os
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from hmmlearn import hmm
from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

### 1) Load & Process the dataset

In [2]:
# Constants
exercise = "squat"
keypoint_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

In [3]:
# Process the JSON files and extract pose data
def process_json_files(data_dir):
    data = []
    labels = []

    # Iterate over the folders in the data directory
    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)

        # Skip non-directory files (e.g., .DS_Store)
        if not os.path.isdir(folder_path):
            continue

        # Determine the label based on whether it's the trainer or trainee folder
        if folder_name == "s08":
            label = "correct"
        else:
            label = "incorrect"

        # Path to the joints3d_25 folder
        joints3d_path = os.path.join(folder_path, "joints3d_25")

        # Iterate over the files in the joints3d_25 folder
        for filename in os.listdir(joints3d_path):
            if filename.endswith(".json") and filename != ".DS_Store":
                file_path = os.path.join(joints3d_path, filename)

                # Open and parse the JSON file
                with open(file_path, "r") as f:
                    json_data = json.load(f)

                # Extract skeleton coordinates for each frame
                for frame_data in json_data["joints3d_25"]:
                    pose_data = []

                    # Extract coordinates for each body part
                    for part_coords in frame_data:
                        pose_data.extend(part_coords)

                    # Append the pose data and label to the lists
                    data.append(pose_data)
                    labels.append(label)

    return data, labels

# Specify the data directory
data_dir = "D:/AI/pos_estimation/train/train"

# Process the JSON files and extract pose data
data, labels = process_json_files(data_dir)

# Print the shape of the data and labels
print("Data shape:", len(data))
print("Labels shape:", len(labels))


Data shape: 20901
Labels shape: 20901


In [4]:
# Process the JSON files and extract pose data
data, labels = process_json_files(data_dir)

In [5]:
# Encode the labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [6]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)

In [7]:
print("Length of X_train:", len(X_train))
print("Length of X_test:", len(X_test))

Length of X_train: 16720
Length of X_test: 4181


In [8]:
# Convert data to tensors
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.int32)

In [9]:
# Reshape the data
X_train = tf.reshape(X_train, shape=(-1, 25, 3))
X_test = tf.reshape(X_test, shape=(-1, 25, 3))

In [10]:
# Normalize the data
X_train /= 255.0
X_test /= 255.0

In [49]:
# Print the shape of train and test data
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (16720, 25, 3)
X_test shape: (4181, 25, 3)


### 2) Feature Extraction with CNN

1) Model

In [11]:
# Build the CNN model
model = tf.keras.Sequential([
    layers.Conv1D(32, 3, activation='relu', input_shape=(25, 3)),
    layers.MaxPooling1D(2),
    layers.Conv1D(64, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')  # For labels/targets we have 2 classes: correct and incorrect
])

In [13]:
# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 23, 32)            320       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 11, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 9, 64)             6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 4, 64)            0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 64)                1

In [17]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
# Train the model and store the training history
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
523/523 [==============================] - 6s 8ms/step - loss: 0.1797 - accuracy: 0.9417 - val_loss: 0.2044 - val_accuracy: 0.9266
Epoch 2/10
523/523 [==============================] - 4s 7ms/step - loss: 0.1613 - accuracy: 0.9453 - val_loss: 0.1574 - val_accuracy: 0.9366
Epoch 3/10
523/523 [==============================] - 4s 7ms/step - loss: 0.1394 - accuracy: 0.9511 - val_loss: 0.1286 - val_accuracy: 0.9584
Epoch 4/10
523/523 [==============================] - 3s 6ms/step - loss: 0.1212 - accuracy: 0.9584 - val_loss: 0.1052 - val_accuracy: 0.9687
Epoch 5/10
523/523 [==============================] - 3s 6ms/step - loss: 0.1013 - accuracy: 0.9660 - val_loss: 0.0930 - val_accuracy: 0.9687
Epoch 6/10
523/523 [==============================] - 4s 7ms/step - loss: 0.0882 - accuracy: 0.9733 - val_loss: 0.0860 - val_accuracy: 0.9720
Epoch 7/10
523/523 [==============================] - 6s 11ms/step - loss: 0.0673 - accuracy: 0.9796 - val_loss: 0.0611 - val_accuracy: 0.9892
Epoch

#### Plot the training results

In [19]:
# Get the training and validation metrics from the history
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

In [20]:
# Plot the training and validation accuracy
plt.plot(train_accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [21]:
# Plot the training and validation loss
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

#### Save the best model

In [22]:
# Specify the path to save the model
path_to_save_model = "D:/AI/pos_estimation/best_models/one"

# Save the model
model.save(path_to_save_model)

INFO:tensorflow:Assets written to: D:/AI/pos_estimation/best_models/one\assets


INFO:tensorflow:Assets written to: D:/AI/pos_estimation/best_models/one\assets


#### Test the above model

In [23]:
import cv2
import numpy as np
import tensorflow as tf

# Load the test video and extract frames
video_path = "D:/AI/pos_estimation/test/test/s02/videos/squat.mp4"  # Replace with the actual path to the test video
#path_to_saved_model = ""

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error opening video file:", video_path)
    exit()

frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

cap.release()

print("Number of frames:", len(frames))

# Process the frames and extract pose data
pose_data = []
# ... code for pose estimation on each frame ...
# Assuming you have a list of pose keypoints for each frame and stored in 'pose_data'

# Prepare the test data
test_data = np.array(pose_data)
test_data = tf.convert_to_tensor(test_data, dtype=tf.float32)
test_data = tf.reshape(test_data, shape=(-1, 25, 3))
test_data /= 255.0

# Load the trained model
model = tf.keras.models.load_model('D:/AI/pos_estimation/best_models/one/saved_model.pb')  # Replace 'path_to_saved_model' with the actual path

# Test the model
predictions = model.predict(test_data)
predicted_labels = np.argmax(predictions, axis=1)

# Print the predicted labels or perform further analysis
print(predicted_labels)


Number of frames: 1736


OSError: Unable to open file (file signature not found)

### Other model

In [31]:
# Constants
exercise = "squat"
keypoint_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]

# Process the JSON files and extract pose data
def process_json_files(data_dir):
    data = []
    labels = []

    # Iterate over the folders in the data directory
    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)

        # Skip non-directory files (e.g., .DS_Store)
        if not os.path.isdir(folder_path):
            continue

        # Determine the label based on whether it's the trainer or trainee folder
        if folder_name == "s08":
            label = "correct"
        else:
            label = "incorrect"

        # Path to the joints3d_25 folder
        joints3d_path = os.path.join(folder_path, "joints3d_25")

        # Iterate over the files in the joints3d_25 folder
        for filename in os.listdir(joints3d_path):
            if filename.endswith(".json") and filename != ".DS_Store":
                file_path = os.path.join(joints3d_path, filename)

                # Open and parse the JSON file
                with open(file_path, "r") as f:
                    json_data = json.load(f)

                # Extract skeleton coordinates for each frame
                for frame_data in json_data["joints3d_25"]:
                    pose_data = []

                    # Extract coordinates for each body part
                    for index in keypoint_indices:
                        part_coords = frame_data[index]
                        pose_data.extend(part_coords)

                    # Append the pose data and label to the lists
                    data.append(pose_data)
                    labels.append(label)

    return data, labels

# Specify the data directory
data_dir = "D:/AI/pos_estimation/train/train"

# Process the JSON files and extract pose data
data, labels = process_json_files(data_dir)

# Encode the labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)

# Convert data to tensors
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.int32)

# Reshape the data
X_train = tf.reshape(X_train, shape=(-1, 25, 3))
X_test = tf.reshape(X_test, shape=(-1, 25, 3))

# Normalize the data
X_train /= 255.0
X_test /= 255.0

# CNN Model
cnn_model = tf.keras.Sequential([
    layers.Conv1D(32, 3, activation='relu', input_shape=(25, 3)),
    layers.MaxPooling1D(2),
    layers.Conv1D(64, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')  # 3 classes: correct, incorrect, almost correct
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
523/523 [==============================] - 6s 7ms/step - loss: 0.3732 - accuracy: 0.8902 - val_loss: 0.3576 - val_accuracy: 0.8852
Epoch 2/10
523/523 [==============================] - 3s 6ms/step - loss: 0.3432 - accuracy: 0.8916 - val_loss: 0.3549 - val_accuracy: 0.8852
Epoch 3/10
523/523 [==============================] - 3s 5ms/step - loss: 0.3372 - accuracy: 0.8916 - val_loss: 0.3566 - val_accuracy: 0.8852
Epoch 4/10
523/523 [==============================] - 3s 5ms/step - loss: 0.3026 - accuracy: 0.8942 - val_loss: 0.2673 - val_accuracy: 0.8974
Epoch 5/10
523/523 [==============================] - 3s 7ms/step - loss: 0.2489 - accuracy: 0.9233 - val_loss: 0.2436 - val_accuracy: 0.9273
Epoch 6/10
523/523 [==============================] - 3s 5ms/step - loss: 0.2315 - accuracy: 0.9294 - val_loss: 0.2299 - val_accuracy: 0.9323
Epoch 7/10
523/523 [==============================] - 3s 5ms/step - loss: 0.2204 - accuracy: 0.9328 - val_loss: 0.2219 - val_accuracy: 0.9357
Epoch 

                                        **********************************

### HMM Model

#### Try-1

In [ ]:
# Extract features from the trained CNN model
train_features = cnn_model.predict(X_train)
test_features = cnn_model.predict(X_test)

# Organize features into sequences for each repetition of the exercise
exercise_sequences_train = []
exercise_sequences_test = []

# Create sequence for training set
current_sequence = []
current_label = y_train[0]
for i in range(len(train_features)):
    if y_train[i] == current_label:
        current_sequence.append(train_features[i])
    else:
        exercise_sequences_train.append(current_sequence)
        current_sequence = [train_features[i]]
        current_label = y_train[i]
exercise_sequences_train.append(current_sequence)

# Create sequence for test set
current_sequence = []
current_label = y_test[0]
for i in range(len(test_features)):
    if y_test[i] == current_label:
        current_sequence.append(test_features[i])
    else:
        exercise_sequences_test.append(current_sequence)
        current_sequence = [test_features[i]]
        current_label = y_test[i]
exercise_sequences_test.append(current_sequence)

# Train an HMM model on the feature sequences
num_states = 3  # Specify the desired number of HMM states

hmm_model = hmm.GaussianHMM(n_components=num_states, covariance_type="diag", n_iter=100)
for sequence in exercise_sequences_train:
    hmm_model.fit(sequence)

# Evaluate the trained HMM model on the test set and calculate performance metrics
predicted_labels = []
for sequence in exercise_sequences_test:
    sequence_likelihoods = []
    for state in range(num_states):
        log_likelihood = hmm_model.score(sequence)
        sequence_likelihoods.append(log_likelihood)
    predicted_label = np.argmax(sequence_likelihoods)
    predicted_labels.append(predicted_label)

accuracy = accuracy_score(y_test, predicted_labels)
print("Accuracy:", accuracy)


#### Try-2

In [19]:
# Extract features from the trained CNN model
train_features = cnn_model.predict(X_train)
test_features = cnn_model.predict(X_test)

# Organize features into sequences for each repetition of the exercise
exercise_sequences_train = []
exercise_sequences_test = []

# Create sequence for training set
current_sequence = []
current_label = y_train[0]
for i in range(len(train_features)):
    if y_train[i] == current_label:
        current_sequence.append(train_features[i])
    else:
        exercise_sequences_train.append(current_sequence)
        current_sequence = [train_features[i]]
        current_label = y_train[i]
exercise_sequences_train.append(current_sequence)

# Create sequence for test set
current_sequence = []
current_label = y_test[0]
for i in range(len(test_features)):
    if y_test[i] == current_label:
        current_sequence.append(test_features[i])
    else:
        exercise_sequences_test.append(current_sequence)
        current_sequence = [test_features[i]]
        current_label = y_test[i]
exercise_sequences_test.append(current_sequence)

# Train an HMM model on the feature sequences
num_states = 3  # Specify the desired number of HMM states

hmm_model = hmm.GaussianHMM(n_components=num_states, covariance_type="diag", n_iter=100)
for sequence in exercise_sequences_train:
    hmm_model.fit(sequence)

# Evaluate the trained HMM model on the test set and calculate performance metrics
predicted_labels = []
for sequence in exercise_sequences_test:
    sequence_likelihoods = []
    for state in range(num_states):
        log_likelihood = hmm_model.score(sequence)
        sequence_likelihoods.append(log_likelihood)
    predicted_label = np.argmax(sequence_likelihoods)
    predicted_labels.append(predicted_label)

accuracy = accuracy_score(y_test, predicted_labels)
print("Accuracy:", accuracy)


131/131 [==============================] - 1s 5ms/step


Model is not converging.  Current: 55.6733104794739 is not greater than 62.73782106764467. Delta is -7.064510588170769
Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Fitting a model with 26 free scalar parameters with only 3 data points will result in a degenerate solution.
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'


ValueError: n_samples=1 should be >= n_clusters=3.

#### Try-3

In [ ]:
# Train an HMM model on the feature sequences
num_states = 3  # Specify the desired number of HMM states

hmm_models = []  # List to store HMM models for each repetition
for sequence in exercise_sequences_train:
    if len(sequence) >= num_states:
        hmm_model = hmm.GaussianHMM(n_components=num_states, covariance_type="diag", n_iter=100)
        hmm_model.fit(sequence)
        hmm_models.append(hmm_model)

# Evaluate the trained HMM models on the test set and calculate performance metrics
predicted_labels = []
for sequence in exercise_sequences_test:
    if len(sequence) >= num_states:
        sequence_likelihoods = []
        for hmm_model in hmm_models:
            log_likelihood = hmm_model.score(sequence)
            sequence_likelihoods.append(log_likelihood)
        predicted_label = np.argmax(sequence_likelihoods)
        predicted_labels.append(predicted_label)

accuracy = accuracy_score(y_test, predicted_labels)
print("Accuracy:", accuracy)


#### Try-4

In [ ]:
# Train an HMM model on the feature sequences
num_states = 3  # Specify the desired number of HMM states

hmm_models = []  # List to store HMM models for each repetition
for sequence in exercise_sequences_train:
    if len(sequence) >= num_states:
        hmm_model = hmm.GaussianHMM(n_components=num_states, covariance_type="diag", n_iter=100, startprob_prior=np.full(num_states, 1/num_states))
        hmm_model.fit(sequence)
        hmm_models.append(hmm_model)

# Evaluate the trained HMM models on the test set and calculate performance metrics
predicted_labels = []
for sequence in exercise_sequences_test:
    if len(sequence) >= num_states:
        sequence_likelihoods = []
        for hmm_model in hmm_models:
            log_likelihood = hmm_model.score(sequence)
            sequence_likelihoods.append(log_likelihood)
        predicted_label = np.argmax(sequence_likelihoods)
        predicted_labels.append(predicted_label)

accuracy = accuracy_score(y_test, predicted_labels)
print("Accuracy:", accuracy)

#### Try-5

In [ ]:
# Train an HMM model on the feature sequences
num_states = 3  # Specify the desired number of HMM states

hmm_models = []  # List to store HMM models for each repetition
for sequence in exercise_sequences_train:
    if len(sequence) >= num_states:
        hmm_model = hmm.GaussianHMM(n_components=num_states, covariance_type="diag", n_iter=100)
        hmm_model.startprob_ = np.full(num_states, 1/num_states)
        hmm_model.fit(sequence)
        hmm_models.append(hmm_model)

# Evaluate the trained HMM models on the test set and calculate performance metrics
predicted_labels = []
for sequence in exercise_sequences_test:
    if len(sequence) >= num_states:
        sequence_likelihoods = []
        for hmm_model in hmm_models:
            log_likelihood = hmm_model.score(sequence)
            sequence_likelihoods.append(log_likelihood)
        predicted_label = np.argmax(sequence_likelihoods)
        predicted_labels.append(predicted_label)

accuracy = accuracy_score(y_test, predicted_labels)
print("Accuracy:", accuracy)


#### Try-6

In [ ]:
# Train an HMM model on the feature sequences
num_states = 3  # Specify the desired number of HMM states

hmm_models = []  # List to store HMM models for each repetition
for sequence in exercise_sequences_train:
    if len(sequence) >= num_states:
        hmm_model = hmm.GaussianHMM(n_components=num_states, covariance_type="diag", n_iter=100)
        hmm_model.startprob_ = np.full(num_states, 1/num_states)
        hmm_model.fit(sequence)
        hmm_model.startprob_ = hmm_model.startprob_ / np.sum(hmm_model.startprob_)  # Normalize start probabilities
        hmm_models.append(hmm_model)

# Evaluate the trained HMM models on the test set and calculate performance metrics
predicted_labels = []
for sequence in exercise_sequences_test:
    if len(sequence) >= num_states:
        sequence_likelihoods = []
        for hmm_model in hmm_models:
            log_likelihood = hmm_model.score(sequence)
            sequence_likelihoods.append(log_likelihood)
        predicted_label = np.argmax(sequence_likelihoods)
        predicted_labels.append(predicted_label)

accuracy = accuracy_score(y_test, predicted_labels)
print("Accuracy:", accuracy)


#### Try-7

In [ ]:
# Extract features from the trained CNN
train_features = model.predict(X_train)
test_features = model.predict(X_test)

# Organize features into sequences for each repetition of the exercise
# Assuming each repetition consists of 5 frames
n_frames_per_repetition = 5
train_sequences = np.split(train_features, len(X_train) // n_frames_per_repetition)
test_sequences = np.split(test_features, len(X_test) // n_frames_per_repetition)

# Train an HMM model
hmm_model = hmm.GaussianHMM(n_components=3, covariance_type="diag")
hmm_model.fit(train_sequences)

# Predict labels using the HMM model
train_predictions = hmm_model.predict(train_sequences)
test_predictions = hmm_model.predict(test_sequences)

# Evaluate the performance of the HMM model
train_accuracy = (train_predictions == y_train.reshape(-1, n_frames_per_repetition)).mean()
test_accuracy = (test_predictions == y_test.reshape(-1, n_frames_per_repetition)).mean()

print("Train accuracy:", train_accuracy)
print("Test accuracy:", test_accuracy)

### Train the CNN using the labeled pose data, with correct and incorrect poses as target labels

In [33]:
# Define the CNN model
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(25, 3)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [34]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [35]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

ValueError: Data cardinality is ambiguous:
  x sizes: 5
  y sizes: 16720
Make sure all arrays contain the same number of samples.

#### Fine-tune the CNN to optimize its performance on pose classification

In [36]:
# Define the CNN model
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(25, 3)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [37]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [38]:
# Define early stopping callback
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

In [39]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

ValueError: Data cardinality is ambiguous:
  x sizes: 5
  y sizes: 16720
Make sure all arrays contain the same number of samples.

2

In [143]:
# Process the JSON files and extract pose data
def process_json_files(data_dir, repetition_count):
    data = []
    labels = []

    # Iterate over the folders in the data directory
    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)

        # Skip non-directory files (e.g., .DS_Store)
        if not os.path.isdir(folder_path):
            continue

        # Determine the label based on whether it's the trainer or trainee folder
        if folder_name == "s08":
            label = "correct"
        else:
            label = "incorrect"

        # Path to the joints3d_25 folder
        joints3d_path = os.path.join(folder_path, "joints3d_25")

        # Iterate over the files in the joints3d_25 folder
        for filename in os.listdir(joints3d_path):
            if filename.endswith(".json") and filename != ".DS_Store":
                file_path = os.path.join(joints3d_path, filename)

                # Open and parse the JSON file
                with open(file_path, "r") as f:
                    json_data = json.load(f)

                # Extract skeleton coordinates for each frame
                frame_count = len(json_data["joints3d_25"])
                if frame_count >= repetition_count:
                    for i in range(repetition_count):
                        pose_data = []
                        start_frame = i * (frame_count // repetition_count)
                        end_frame = (i + 1) * (frame_count // repetition_count)

                        # Extract coordinates for each body part in the specified frame range
                        for frame_data in json_data["joints3d_25"][start_frame:end_frame]:
                            for part_coords in frame_data:
                                pose_data.extend(part_coords)

                        # Append the pose data and label to the lists
                        data.append(pose_data)
                        labels.append(label)

    return data, labels


# Specify the data directory and repetition count
data_dir = "D:/AI/pos_estimation/train/train"
repetition_count = 5

# Process the JSON files and extract pose data
data, labels = process_json_files(data_dir, repetition_count)

# Print the shape of the data and labels
print("Data shape:", len(data))
print("Labels shape:", len(labels))

Data shape: 80
Labels shape: 80


In [50]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the CNN model
def build_cnn_model(input_shape):
    model = tf.keras.Sequential()

    # Convolutional layers
    model.add(layers.Conv1D(32, 3, activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Conv1D(64, 3, activation='relu'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Conv1D(128, 3, activation='relu'))
    model.add(layers.MaxPooling1D(2))

    # Flatten layer
    model.add(layers.Flatten())

    # Dense layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Output layer for binary classification

    return model

# Specify the input shape based on your data
input_shape = (25, 3)  # Assuming each sequence has 25 frames with 3 coordinates (x, y, z)

# Build the CNN model
model = build_cnn_model(input_shape)

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_11 (Conv1D)          (None, 23, 32)            320       
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 11, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_12 (Conv1D)          (None, 9, 64)             6208      
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 4, 64)            0         
 g1D)                                                            
                                                                 
 conv1d_13 (Conv1D)          (None, 2, 128)            24704     
                                                                 
 max_pooling1d_13 (MaxPoolin  (None, 1, 128)          

In [51]:
# Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

loss, accuracy = model.evaluate(X_val, y_val)
print("Validation loss:", loss)
print("Validation accuracy:", accuracy)



NameError: name 'X_val' is not defined

### 3) Temporal Modeling with HMM

In [132]:
# Extract the features from the CNN for each frame of the pose data:
from tensorflow.keras.models import Model

# Define the feature extraction model
feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)

# Extract features from the CNN for each frame of pose data
X_train_features = feature_extractor.predict(X_train)
X_test_features = feature_extractor.predict(X_test)


131/131 [==============================] - 0s 2ms/step


In [ ]:
# Assuming you have identified the start and end indices for each repetition
repetition_indices = [(start1, end1), (start2, end2), ...]

# Initialize empty lists to store the organized feature sequences
organized_train_sequences = []
organized_test_sequences = []

# Organize the features into sequences for each repetition in the training data
for start, end in repetition_indices:
    sequence = X_train_features[start:end]
    organized_train_sequences.append(sequence)

# Organize the features into sequences for each repetition in the testing data
for start, end in repetition_indices:
    sequence = X_test_features[start:end]
    organized_test_sequences.append(sequence)
